# Fluid Flow only

## Parabolic Slider

We want to investigate the pressure and flow induced by a 2D parabolic slider ([YAML File](./examples/parabolic_slider_2d.yaml)).



Features of this use case:
- $U = 50$ m/s
- $V = 0$ m/s
- $h_{\mathrm{min}} = 1 \cdot 10^{-5}$ m
- $h_{\mathrm{max}} = 6.6 \cdot 10^{-5}$ m
- $p_{\mathrm{BC}} = 1$ bar
- $\frac{\partial j_x}{x}_{\mathrm{BC}} = 0$
- $\frac{\partial j_y}{y}_{\mathrm{BC}} = 0$

In [ ]:
from GaPFlow.problem import Problem

problem = Problem.from_yaml("examples/parabolic_slider_2d.yaml")
problem.plot_topo()

In [ ]:
problem.run()

In [ ]:
problem.plot()

## Couette Flow

initial zero flow, building up by U

In [ ]:
from GaPFlow.problem import Problem
import numpy as np

problem = Problem.from_yaml("examples/couette_2d.yaml")
problem.q[1] = 0. # Set jx = 0 everywhere
problem.run()

print(np.max(problem.q[2]))  # Max jy
print(np.min(problem.q[2]))  # Min jy

problem.animate()

# Energy

## Diffusion

Dynamic simulation of 2D heat diffusion with comparison to the analytic solution ([YAML File](./examples/temp_diffusion_2d.yaml)).

Setup:
- Dirichlet temperature boundary conditions in x: T=0 at both ends
- Periodic boundary conditions in y
- Initial temperature: sine profile T(x,0) = T_max * sin(π*x_norm) constant over y
- No wall movement (U=0, V=0), pure diffusion

Since the initial condition and boundary conditions are uniform in y, and y has periodic BCs, the solution should match the 1D analytic solution:

$$T(x,t) = T_{max} \cdot e^{-\lambda^2 t} \cdot \sin\left(\pi \cdot x_{norm}\right)$$

where $\lambda = \sqrt{\alpha} \cdot \frac{\pi}{L_{eff}}$ and $\alpha = \frac{k}{c_v \rho}$.

In [ ]:
# 2D Temperature diffusion
import numpy as np
from GaPFlow.problem import Problem

# Store temperature history for comparison
T_history = {'time': [], 'T': []}

problem = Problem.from_yaml("examples/temp_diffusion_2d.yaml")

def track_temperature():
    dt = problem.numerics['dt']
    t = problem.step * dt
    T_history['time'].append(t)
    # Extract T along x at mid-y (should be constant in y)
    T_history['T'].append(problem.energy.temperature[:, problem.grid['Ny']//2 + 1].copy())

problem.add_callback(track_temperature)
problem.run()

In [ ]:
# Compare with analytic solution
import sys; sys.path.insert(0, '.')
from utils.plotting import animate_comparison  # type: ignore
from GaPFlow.utils import heat_equation_1d

# Get grid parameters
L = problem.grid['Lx']
Nx = problem.grid['Nx']
dx = problem.grid['dx']
cv = problem.energy_spec['cv']
k = problem.energy_spec['k']
rho = problem.prop['rho0']
T_max = problem.energy_spec['T0'][2]

# Cell centers including ghost cells: -dx/2, dx/2, 3dx/2, ..., Lx+dx/2
x_vec = np.arange(Nx + 2) * dx - dx / 2
t_vec = np.array(T_history['time'])

# For half_sine_ghost: T = sin(pi * (x + dx/2) / (Lx + dx))
# Zeros at ghost cell centers: x = -dx/2 and x = Lx + dx/2
# This matches where FEM 2D enforces Dirichlet BC
L_eff = L + dx
x_shifted = x_vec + dx / 2  # shift so x_shifted=0 at x=-dx/2

T_analytic = heat_equation_1d(L_eff, cv, k, rho, T_max, x_shifted, t_vec)
T_numeric = np.array(T_history['T'])

animate_comparison(x_vec, t_vec, T_numeric, T_analytic, L)

## Advection

Dynamic simulation of 2D temperature advection with a sine initial profile ([YAML File](./examples/temp_advection_2d.yaml)).

Setup:
- Periodic boundary conditions in x and y
- Wall motion U > 0 drives Couette flow which advects the temperature
- Initial temperature: half-sine profile in x, constant in y
- No thermal diffusion (k=0)

The temperature wave travels with the average flow velocity. For Couette flow with wall velocity U, the average velocity is U/2. With periodic boundaries, the sine profile wraps around the domain.

Since the setup is uniform in y (periodic BC, constant initial T in y), the 2D solution should match 1D behavior.

Terms used:
- `R31x`, `R31y`: Temperature convection (advection with velocity field)
- `R3T`: Energy time derivative
- Mass and momentum terms for density and velocity fields

In [ ]:
# 2D Temperature advection
import numpy as np
from GaPFlow.problem import Problem

problem = Problem.from_yaml("examples/temp_advection_2d.yaml")

# Store temperature history for animation
T_history = {'time': [], 'T': []}

def track_T():
    T_history['time'].append(problem.simtime)
    # Extract T along x at mid-y (should be constant in y)
    T_history['T'].append(problem.energy.temperature[:, problem.grid['Ny']//2 + 1].copy())

problem.add_callback(track_T)
problem.run()

In [ ]:
# Animate temperature advection
import sys; sys.path.insert(0, '.')
from utils.plotting import animate_advection  # type: ignore

# Get grid parameters
L = problem.grid['Lx']
Nx = problem.grid['Nx']
dx = problem.grid['dx']
U = problem.geo['U']

# For Couette flow, average velocity is U/2
U_avg = U / 2

# Cell centers including ghost cells
x_vec = np.arange(Nx + 2) * dx - dx / 2
t_vec = np.array(T_history['time'])
T_numeric = np.array(T_history['T'])

animate_advection(x_vec, t_vec, T_numeric, L, U_avg)

# Elastic Deformation